In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import gensim

from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

# Any results you write to the current directory are saved as output.

['ks-projects-201801.csv', 'ks-projects-201612.csv']


In [27]:
df = pd.read_csv('../input/ks-projects-201801.csv')
df.head(10)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.00,failed,0,GB,0.00,0.00,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.00,failed,15,US,100.00,2421.00,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.00,failed,3,US,220.00,220.00,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.00,failed,1,US,1.00,1.00,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.00,canceled,14,US,1283.00,1283.00,19500.00
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.00,successful,224,US,52375.00,52375.00,50000.00
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.00,successful,16,US,1205.00,1205.00,1000.00
7,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17,25000.0,2016-02-01 20:05:12,453.00,failed,40,US,453.00,453.00,25000.00
8,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29,125000.0,2014-04-24 18:14:43,8233.00,canceled,58,US,8233.00,8233.00,125000.00
9,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10,65000.0,2014-07-11 21:55:48,6240.57,canceled,43,US,6240.57,6240.57,65000.00


In [28]:
names_data = df.loc[:,'name']
names_data.head()

0                      The Songs of Adelaide & Abullah
1        Greeting From Earth: ZGAC Arts Capsule For ET
2                                       Where is Hank?
3    ToshiCapital Rekordz Needs Help to Complete Album
4    Community Film Project: The Art of Neighborhoo...
Name: name, dtype: object

In [29]:
def read_questions(row,column_name):
    return gensim.utils.simple_preprocess(str(row[column_name]).encode('utf-8'))

documents = []
for index, row in df.iterrows():
    if row["state"] == 'successful':
        documents.append(read_questions(row,"name"))

print(type(documents))

<class 'list'>


In [19]:
#simplified_df = df.transform()
idx = np.logical_or(df['state']=='successful',df['state']=='failed')
docs = df.loc[idx,'name'].values.astype('U')
feature_2 = df.loc[idx,'main_category'].values.astypr('U')
feature_3 = df.loc[idx,'goal'].values.astype('U')

targets = df.loc[idx,'state'].values

output = np.ones(len(targets),dtype='bool')
for i in range(0,len(targets)):
    if(targets[i]=='failed'):
        output[i] = False

# Shows fraction of inputs that have been successful out of the total dataset.
print(sum(output)/len(targets))

#print(len(df))

0.4038772895153388


In [30]:
X_train, X_test, y_train, y_test = train_test_split(docs, output, 
                                                          test_size = 0.20, random_state = 12)

In [31]:
# Add the argument 'ngram_range' later to see if the efficiency improves with bigrams - no improvements whatsoever with bigrams in case of simple regressors
vect = CountVectorizer(min_df=1)

X_train_counts = vect.fit_transform(X_train)
print(vect.vocabulary_['love'])


60497


In [33]:
feature_transformer = TfidfTransformer()
X_train_tfidf = feature_transformer.fit_transform(X_train_counts)

In [34]:
X_train_tfidf.shape

(265340, 116123)

In [35]:
X_test_counts = vect.transform(X_test)
X_test_tfidf = feature_transformer.transform(X_test_counts)
print(X_test_tfidf.shape)

(66335, 116123)


In [36]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [37]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test_tfidf)
print(accuracy_score(y_test, y_pred))

0.6470038441245195


In [38]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
lg.fit(X_train_tfidf,y_train)
y_pred = lg.predict(X_test_tfidf)
print(accuracy_score(y_test,y_pred))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6541946182256727


In [39]:

model = gensim.models.Word2Vec(size=150, window=10, min_count=2, sg=1, workers=10)
model.build_vocab(documents)
model.train(sentences=documents, total_examples=len(documents), epochs=model.iter)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.


(3014070, 3694445)

In [40]:
word_vectors = model.wv
count = 0
for word in word_vectors.vocab:
    if count<10:
        print(word)
        count += 1
    else:
        break

monarch
espresso
bar
support
solar
roasted
coffee
green
energy
co


In [41]:
len(word_vectors.vocab)

27651

In [42]:
#print(model.wv["education"])

In [43]:
model.wv.most_similar(positive=['dating'],topn=20)

[('serial', 0.9423626065254211),
 ('relationships', 0.9373728036880493),
 ('ya', 0.9323734641075134),
 ('murder', 0.931825578212738),
 ('superhero', 0.929128110408783),
 ('allegiance', 0.926696240901947),
 ('otome', 0.9252296686172485),
 ('genre', 0.9247218370437622),
 ('paranormal', 0.9228708148002625),
 ('destroy', 0.9208618998527527),
 ('spider', 0.9202732443809509),
 ('identity', 0.9200851321220398),
 ('cop', 0.9187102317810059),
 ('extreme', 0.914598286151886),
 ('fantastical', 0.9127919673919678),
 ('bigfoot', 0.912534236907959),
 ('satire', 0.9122405648231506),
 ('crime', 0.9120154976844788),
 ('villain', 0.9115597605705261),
 ('economics', 0.9114870429039001)]

In [44]:
model.wv.most_similar(positive=['love'],topn=20)

[('hope', 0.819797158241272),
 ('telling', 0.8050882816314697),
 ('loss', 0.7976760864257812),
 ('tell', 0.7853730916976929),
 ('faith', 0.7824855446815491),
 ('sad', 0.7782013416290283),
 ('told', 0.7770155072212219),
 ('healing', 0.7756072282791138),
 ('times', 0.7745287418365479),
 ('same', 0.7712809443473816),
 ('birth', 0.7697397470474243),
 ('tall', 0.7685649394989014),
 ('having', 0.7683750987052917),
 ('hiv', 0.7681182026863098),
 ('daddy', 0.7675319910049438),
 ('illness', 0.7656395435333252),
 ('found', 0.765511155128479),
 ('giving', 0.7648804187774658),
 ('loving', 0.7636110186576843),
 ('bedtime', 0.7634231448173523)]

In [45]:
model.wv.most_similar(positive=['gaming'],topn=20)

[('tabletop', 0.9032766222953796),
 ('miniature', 0.8941982984542847),
 ('rpgs', 0.887352466583252),
 ('gamers', 0.8863370418548584),
 ('terrain', 0.8814631700515747),
 ('maps', 0.8791557550430298),
 ('printable', 0.8775759339332581),
 ('combat', 0.8771548867225647),
 ('module', 0.8764572739601135),
 ('tiles', 0.8756310343742371),
 ('soundtracks', 0.8727599382400513),
 ('tile', 0.8710875511169434),
 ('retro', 0.8693990707397461),
 ('modules', 0.8687559366226196),
 ('dungeon', 0.8671344518661499),
 ('wargaming', 0.8659077286720276),
 ('tokens', 0.8652111887931824),
 ('gamer', 0.8646261096000671),
 ('command', 0.8636422753334045),
 ('props', 0.8635660409927368)]

In [46]:
model.wv.most_similar(positive=['canada'],topn=20)

[('germany', 0.9403465390205383),
 ('colombia', 0.9345265030860901),
 ('california', 0.9326885938644409),
 ('england', 0.921557605266571),
 ('sweden', 0.9191785454750061),
 ('hawaii', 0.9189077019691467),
 ('paris', 0.918703019618988),
 ('turkey', 0.9133180379867554),
 ('spain', 0.9117759466171265),
 ('southwest', 0.9101532697677612),
 ('georgia', 0.9093400835990906),
 ('veggie', 0.9075808525085449),
 ('argentina', 0.905782163143158),
 ('mercies', 0.9055565595626831),
 ('cleveland', 0.9051097631454468),
 ('pond', 0.9048838019371033),
 ('wales', 0.9037802815437317),
 ('europe', 0.9025857448577881),
 ('campfire', 0.9025477766990662),
 ('ghana', 0.9018175601959229)]

In [ ]:
# Predicting success rate just using 'names' tag and see how much success we achieve


